In [ ]:
import logging
import warnings

import mne
import xarray as xr
import yaml
from megspikes.pipeline import aspire_alphacsc_pipeline
from megspikes.visualization.report import report_detection, report_atoms_library

from utils.utils import setup_case_manager

warnings.filterwarnings("ignore", category=DeprecationWarning)
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.debug("test debug")
logging.info("test info")

from sklearn import set_config
set_config(display='diagram')
%load_ext autoreload
%autoreload 2


In [ ]:
case = setup_case_manager(1)

detection_results = xr.open_dataset(case.dataset)

In [ ]:
# Inspect dataset

detection_results

In [ ]:
# explore dataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr



In [ ]:
# the number of events in the libraries equals the alpha csc dection in mag and grad
%matplotlib qt 
events_lib_tot  = np.sum(detection_results.alphacsc_atoms_library_properties[0].values)
events_lib_grad = np.sum(detection_results.detection_properties.loc[dict(sensors = 'grad', detection_property = 'alphacsc_detection')].values)
events_lib_mag  = np.sum(detection_results.detection_properties.loc[dict(sensors = 'mag', detection_property = 'alphacsc_detection')].values) 

print("the total number of events is " + str(events_lib_tot) + " = " + str(events_lib_grad) + " (grad) " +  str(events_lib_mag) + " (mag) ")

In [ ]:
# compare z hat with atoms time stamps

%matplotlib qt 

from scipy import signal, stats
import matplotlib.lines as lines

run  = 0 # there is only 1
sens = 0 # grad 
atom = 0 # N atom 
z_hat = detection_results.alphacsc_z_hat[run][sens][atom]

# ev1
ev1 = np.nonzero(detection_results.detection_properties.loc[dict( detection_property= 'alphacsc_detection', sensors = 'grad')].values)[1]
alphacsc_atom = detection_results.detection_properties.loc[dict( detection_property = 'alphacsc_atom', sensors = 'grad')].values[0]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
 

# plot zhat and superimpose events time stamp
# ( consider that the events are searched at zhat peak immediately preceding the ica detections:
# the zhat peak set the beginning of a 0.5 s window where the spike appears, therefore an alignment
# must be done for the events clustering. see detection.SelectAlphacscEvents for details)

ax1 = plt.subplot(2,1,1)
ax1.plot(detection_results.time, z_hat)
ax1.stem(ev1/200, np.ones(len(ev1))*7e-10,'r')
ax1.set_title('z_hat') 

# add threshold as horizontal bar
z_threshold = 2.5 # default value
z_mad = stats.median_abs_deviation(z_hat[z_hat > 0])
threshold = np.median(z_hat[z_hat > 0]) + z_mad*z_threshold 
ax1.add_artist(lines.Line2D([0, detection_results.time[-1]],[threshold, threshold],color ='g'))

ax2 = plt.subplot(2,1,2, sharex=ax1)
ax2.stem(ev1/200,alphacsc_atom[ev1])
ax2.set_title('which atom') 


